# 高阶函数

## map/reduce函数

map()函数接受两个参数，一个是func，一个是Iterable，map将传入的函数依次作用到序列的每个元素，并把结果作为一个新的Iterator返回。<br>
比如我们有一个函数f(x)=x2，要把这个函数作用在一个list [1, 2, 3, 4, 5, 6, 7, 8, 9]上，就可以用map()实现如下：<br>

![](https://cdn.webxueyuan.com/cdn/files/attachments/0013879622109990efbf9d781704b02994ba96765595f56000/0)

In [1]:
def f(x):
    return x*x

In [2]:
m=map(f,[1,2,3,4,5,6,7,8,9])

In [4]:
from collections import Iterator
isinstance(m,Iterator)

True

In [5]:
list(m)

[1, 4, 9, 16, 25, 36, 49, 64, 81]

map()传入的第一个参数是f，即函数对象本身。由于结果r是一个Iterator，Iterator是惰性序列，因此通过list()函数让它把整个序列都计算出来并返回一个list。

map()作为高阶函数，事实上它把运算规则抽象了，因此，我们不但可以计算简单的f(x)=x2，还可以计算任意复杂的函数，比如，把这个list所有数字转为字符串：

In [6]:
list(map(str,[1,2,3,4,5,6]))

['1', '2', '3', '4', '5', '6']

reduce把一个函数作用在一个序列[x1, x2, x3, ...]上，这个函数必须接收两个参数，reduce把结果继续和序列的下一个元素做累积计算，其效果就是：

```
reduce(f, [x1, x2, x3, x4]) = f(f(f(x1, x2), x3), x4)
```

如果要把序列[1, 3, 5, 7, 9]变换成整数13579，reduce就可以派上用场：

In [7]:
from functools import reduce
def fn(x,y):
    return x*10+y
reduce(fn,[1,2,3,4,5])

12345

如果考虑到字符串str也是一个序列，对上面的例子稍加改动，配合map()，我们就可以写出把str转换为int的函数：

In [8]:
def str2int(s):
    def fn(x,y):
        return x*10+y
    def char2num(s):
        return {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9}[s]
    return reduce(fn, map(char2num, s))

In [9]:
str2int('1234')

1234

## filter 函数

Python内建filter()函数用于过滤序列。<br>
和map()类似，filter()也接收一个函数和一个序列。和map()不同的是，filter()把传入的函数依次作用于每个元素，然后根据返回值是True还是False决定保留还是丢弃该元素。<br>
filter()返回的也是一个Iterator。

In [10]:
def is_odd(n):
    return n%2==1

In [11]:
list(filter(is_odd,[1,2,3,4,5,6,8,9]))

[1, 3, 5, 9]

### filter求素数

计算素数的一个方法是埃氏筛法，它的算法理解起来非常简单：

首先，列出从2开始的所有自然数，构造一个序列：

2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...

取序列的第一个数2，它一定是素数，然后用2把序列的2的倍数筛掉：

3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...

取新序列的第一个数3，它一定是素数，然后用3把序列的3的倍数筛掉：

5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...

取新序列的第一个数5，然后用5把序列的5的倍数筛掉：

7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...

不断筛下去，就可以得到所有的素数。

用Python来实现这个算法，可以先构造一个从3开始的奇数序列：

In [23]:
def _odd_iter():
    n = 1
    while True:
        n = n + 2
        yield n
# 注意这是一个生成器，并且是一个无限序列。

In [24]:
# 筛选函数
def _not_divisible(n):
    return lambda x: x % n > 0

In [25]:
# 定义一个生成器，不断返回下一个素数：
def primes():
    yield 2
    it = _odd_iter() # 初始序列
    while True:
        n = next(it) # 返回序列的第一个数
        yield n
        it = filter(_not_divisible(n), it) # 构造新序列

这个生成器先返回第一个素数2，然后，利用filter()不断产生筛选后的新的序列。

由于primes()也是一个无限序列，所以调用时需要设置一个退出循环的条件：

In [27]:
# 打印1000以内的素数:
for n in primes():
    if n < 10:
        print(n)
    else:
        break

2
3
5
7


## sorted函数

排序也是在程序中经常用到的算法。无论使用冒泡排序还是快速排序，排序的核心是比较两个元素的大小。如果是数字，我们可以直接比较，但如果是字符串或者两个dict呢？直接比较数学上的大小是没有意义的，因此，比较的过程必须通过函数抽象出来。

In [28]:
sorted([36, 5, -12, 9, -21])

[-21, -12, 5, 9, 36]

In [29]:
sorted([36, 5, -12, 9, -21], key=abs)

[5, 9, -12, -21, 36]

In [30]:
sorted(['bob', 'about', 'Zoo', 'Credit'], key=str.lower)

['about', 'bob', 'Credit', 'Zoo']

In [31]:
sorted(['bob', 'about', 'Zoo', 'Credit'], key=str.lower, reverse=True)

['Zoo', 'Credit', 'bob', 'about']